# Analyzing the logs

In [1]:
# Run from the top of the repository
%cd ..

/raid/home/pbachant/calkit/boom-paper


In [2]:
import pandas as pd
import polars as pl
import yaml
import itertools
import os

with open("calkit.yaml") as f:
    calkit_config = yaml.safe_load(f)

# Load the BOOM and Kowalski configurations
boom_params = {}
for i in calkit_config["pipeline"]["stages"]["benchmark-boom"]["iterate_over"]:
    boom_params[i["arg_name"]] = i["values"]

kowalski_params = {}
for i in calkit_config["pipeline"]["stages"]["benchmark-kowalski"][
    "iterate_over"
]:
    kowalski_params[i["arg_name"]] = i["values"]


# Form the matrix of parameters for each
param_names = list(boom_params.keys())
param_vals = list(boom_params.values())
vals_product = list(itertools.product(*param_vals))
results_boom = []

for vals in vals_product:
    current_params = dict(zip(param_names, vals))
    boom_config = (
        f"na={current_params['n_alert_workers']}-"
        f"nml={current_params['n_ml_workers']}-"
        f"nf={current_params['n_filter_workers']}"
    )
    print(f"Analyzing config: {boom_config}")
    boom_consumer_log_fpath = f"logs/boom-{boom_config}/consumer.log"
    boom_scheduler_log_fpath = f"logs/boom-{boom_config}/scheduler.log"
    # To calculate BOOM wall time, take first timestamp from the consumer log
    # as the start and the last timestamp of the scheduler as the end
    with open(boom_consumer_log_fpath) as f:
        line = f.readline()
        t1_b = pd.to_datetime(
            line.split()[2].replace("\x1b[2m", "").replace("\x1b[0m", "")
        )
    with open(boom_scheduler_log_fpath) as f:
        lines = f.readlines()
        line = lines[-3]
        t2_b = pd.to_datetime(
            line.split()[2].replace("\x1b[2m", "").replace("\x1b[0m", "")
        )
    current_params["start_time"] = t1_b
    current_params["end_time"] = t2_b
    boom_wall_time = t2_b - t1_b
    print(f"BOOM wall time: {boom_wall_time.total_seconds() / 60:.2f} minutes")
    results_boom.append(current_params)

df_boom = pl.DataFrame(results_boom).with_columns(
    (pl.col("end_time") - pl.col("start_time"))
    .dt.total_seconds()
    .alias("wall_time_s")
)

# Kowalski only has one parameter name, so we don't need the product
param_name = list(kowalski_params.keys())[0]
param_vals = kowalski_params[param_name]
results_kowalski = []

for val in param_vals:
    res = {param_name: val}
    kowalski_config = f"n={val}"
    print(f"Analyzing Kowalski config: {kowalski_config}")
    # To calculate Kowalski wall time, just use the supervisord logs
    with open(f"logs/kowalski-{kowalski_config}/supervisord.log") as f:
        lines = f.readlines()
        for line in lines:
            if "alert-broker-ztf entered RUNNING state" in line:
                t1_k = pd.to_datetime("T".join(line.split()[:2]))
                break
        for line in lines:
            if "received SIGTERM indicating exit request" in line:
                t2_k = pd.to_datetime("T".join(line.split()[:2]))
                break
    # Print the total time of each, in minutes
    kowalski_wall_time = t2_k - t1_k  # type: ignore
    res["start_time"] = t1_k
    res["end_time"] = t2_k
    results_kowalski.append(res)
    print(
        f"Kowalski wall time: {kowalski_wall_time.total_seconds() / 60:.2f} "
        "minutes"
    )

df_kowalski = pl.DataFrame(results_kowalski).with_columns(
    (pl.col("end_time") - pl.col("start_time"))
    .dt.total_seconds()
    .alias("wall_time_s")
)

# Write results to file for later analysis
os.makedirs("results", exist_ok=True)
df_boom.write_csv("results/boom.csv")
df_kowalski.write_csv("results/kowalski.csv")

# Calculate throughput factor from a specific config for each
boom_wall_time = df_boom.filter(
    (pl.col("n_alert_workers") == 3)
    & (pl.col("n_ml_workers") == 3)
    & (pl.col("n_filter_workers") == 1)
).select("wall_time_s").row(0)[0]
kowalski_wall_time = (
    df_kowalski.filter(pl.col("n_workers") == 7)
    .select(pl.col("wall_time_s"))
    .row(0)[0]
)

boom_throughput_factor = kowalski_wall_time / boom_wall_time
print(f"BOOM throughput factor: {boom_throughput_factor:.1f}")
print(df_boom)
print(df_kowalski)

Analyzing config: na=1-nml=3-nf=1
BOOM wall time: 3.39 minutes
Analyzing config: na=1-nml=3-nf=3
BOOM wall time: 3.41 minutes
Analyzing config: na=1-nml=6-nf=1
BOOM wall time: 3.35 minutes
Analyzing config: na=1-nml=6-nf=3
BOOM wall time: 3.40 minutes
Analyzing config: na=1-nml=9-nf=1
BOOM wall time: 3.33 minutes
Analyzing config: na=1-nml=9-nf=3
BOOM wall time: 3.36 minutes
Analyzing config: na=3-nml=3-nf=1
BOOM wall time: 1.26 minutes
Analyzing config: na=3-nml=3-nf=3
BOOM wall time: 1.26 minutes
Analyzing config: na=3-nml=6-nf=1
BOOM wall time: 0.98 minutes
Analyzing config: na=3-nml=6-nf=3
BOOM wall time: 1.17 minutes
Analyzing config: na=3-nml=9-nf=1


BOOM wall time: 1.17 minutes
Analyzing config: na=3-nml=9-nf=3
BOOM wall time: 1.17 minutes
Analyzing config: na=6-nml=3-nf=1
BOOM wall time: 1.26 minutes
Analyzing config: na=6-nml=3-nf=3
BOOM wall time: 1.29 minutes
Analyzing config: na=6-nml=6-nf=1
BOOM wall time: 0.76 minutes
Analyzing config: na=6-nml=6-nf=3
BOOM wall time: 0.76 minutes
Analyzing config: na=6-nml=9-nf=1
BOOM wall time: 0.57 minutes
Analyzing config: na=6-nml=9-nf=3
BOOM wall time: 0.57 minutes


Analyzing Kowalski config: n=3
Kowalski wall time: 13.70 minutes
Analyzing Kowalski config: n=6
Kowalski wall time: 7.60 minutes
Analyzing Kowalski config: n=7
Kowalski wall time: 7.53 minutes
Analyzing Kowalski config: n=9
Kowalski wall time: 5.00 minutes
BOOM throughput factor: 6.0
shape: (18, 6)
┌─────────────────┬──────────────┬─────────────────┬────────────────┬────────────────┬─────────────┐
│ n_alert_workers ┆ n_ml_workers ┆ n_filter_worker ┆ start_time     ┆ end_time       ┆ wall_time_s │
│ ---             ┆ ---          ┆ s               ┆ ---            ┆ ---            ┆ ---         │
│ i64             ┆ i64          ┆ ---             ┆ datetime[μs,   ┆ datetime[μs,   ┆ i64         │
│                 ┆              ┆ i64             ┆ UTC]           ┆ UTC]           ┆             │
╞═════════════════╪══════════════╪═════════════════╪════════════════╪════════════════╪═════════════╡
│ 1               ┆ 3            ┆ 1               ┆ 2025-07-26 13: ┆ 2025-07-26 14: ┆ 203    

In [3]:
# TODO: Put actual results here
kafka_ingest_strate_mbps = "XXXXX"
kafka_ingest_factor = "XXXXX"

template = r"""% This file was automatically generated; edits will be overwritten!
\newcommand{\boomthroughputfactor}{BOOM_THROUGHPUT_FACTOR}
\newcommand{\kakfaingestratembps}{KAFKA_INGEST_STRATE_MBPS}
\newcommand{\kakfaingestfactor}{KAFKA_INGEST_FACTOR}
"""

# This might be better with an f-string, but this is TeX and who wants to deal
# with escaping all the braces?
template = (
    template.replace(
        "BOOM_THROUGHPUT_FACTOR", f"{round(boom_throughput_factor):1d}"
    )
    .replace("KAFKA_INGEST_STRATE_MBPS", kafka_ingest_strate_mbps)
    .replace("KAFKA_INGEST_FACTOR", kafka_ingest_factor)
)

with open("paper/results.tex", "w") as f:
    f.write(template)